In [1]:
import scipy.io as sio
import torch
from torch. utils.data import TensorDataset, DataLoader
import numpy as np
import h5py
from helper_BER import helperLinearInterp, helperEqualX ,linear_extrapolation_matrix
from helper.utils import minmaxScaler, deMinMax, calNMSE


In [2]:
import sys
import os
# Get the parent directory of the current folder
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the parent directory to sys.path so Python can find the helper folder
sys.path.append(parent_dir)

# to load models
from helper_BER import helperLoadModels

from train.Pix2Pix.helperPix2Pix.discriminator_model import Discriminator
from train.Pix2Pix.helperPix2Pix.generator_model import Generator, Generator_FineTune
from helper.utils import CNN_Est2, CNN_Est
from helper.utils_transfer import FineTuneModel2

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [7]:
domain = "mapBased"

for snr in np.arange(-25, 35, 5):  # (-25, 35, 5)
    print(f"snr {snr}/30")
    
    # Load Models
    LS_CNN_trained, LI_CNN_trained, LS_GAN_trained, LI_GAN_trained, LS_CNN_transfer, LI_CNN_transfer, LS_GAN_transfer, LI_GAN_transfer = helperLoadModels(device, snr)

    # Load the .mat file
    if domain == "DeepMIMO":
        row = '3500_3516'
        dataFile = '../../DeepMIMOv2/DeepMIMO_Data/Static_BS16/freq_symb_1ant_612sub_ver4/Gan_' + str(snr) +'_dBOutdoor1_3p4_1ant_612subcs_Row_' + str(row) + '.mat'
        mat_data = h5py.File(dataFile, 'r')     # load Y, H, N
        
    elif domain == "mapBased":
        dataFile = '../../CDL Customization/Data/ver39_/' + str(snr) + 'dB/1_mapBaseData.mat'
        mat_data = h5py.File(dataFile, 'r')     # load Y, H, N
        
    H_data = np.array(mat_data['H_data'])                # Shape: (N, 2, 612, 14 )
    H_linear_data = np.array(mat_data['H_linear_data'])  
        # H_equalized_data = np.array(mat_data['H_equalized_data'])  
    Y_data = np.array(mat_data['Y_data'])                
    Noise = np.array(mat_data['Noise'])                  

    mat_data2 = sio.loadmat('../../DeepMIMOv2/DeepMIMO_Data/TransmitedGrid.mat') # load X (transmitted grid)
    X = np.transpose(mat_data2['X'], (3, 2, 1, 0)) 
    
    # 
    num_samples = H_data.shape[0]

    # Randomly select {num_samples} unique indices from 10000 samples of X
    random_indices = np.random.choice(X.shape[0], num_samples, replace=False)
    # Extract the selected samples
    X_data = X[random_indices, :, :, :]  # Shape: (14, 612, 2, {num_samples})
    
    # Convert to PyTorch tensors
    # Assuming complex data; split into real and imaginary if needed
    H_data = torch.tensor(H_data, dtype=torch.float32).to(device)
    # H_linear_data = torch.tensor(H_linear_data, dtype=torch.float32).to(device)
    # H_equalized_data = torch.tensor(H_equalized_data, dtype=torch.float32).to(device)
    Y_data = torch.tensor(Y_data, dtype=torch.float32).to(device)
    Noise = torch.tensor(Noise, dtype=torch.float32).to(device)
    X_data = torch.tensor(X_data, dtype=torch.float32).to(device)
    
    pilot = sio.loadmat('../../DeepMIMOv2/DeepMIMO_Data/pilot_value.mat')
    pilot_Indices = pilot['pilots_idx']
    pilot_Symbols = pilot['pilot_tx']

    pilot_Indices = torch.tensor(pilot_Indices, dtype=torch.int64).to(device)
    pilot_Symbols = torch.tensor(pilot_Symbols, dtype=torch.complex64).to(device)

    # Create TensorDataset
    dataset = TensorDataset(H_data, X_data, Noise)

    # Create DataLoader
    batch_size = 32
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Initialize an empty tensor on CUDA
    X_orig = np.empty((0, H_data.shape[2], H_data.shape[3]))  # Shape (0, 612, 14)
    X_LS_LI = np.empty((0, H_data.shape[2], H_data.shape[3]))
    
    X_LI_CNN_trained = np.empty((0, H_data.shape[2], H_data.shape[3]))  # Shape (0, 612, 14)
    X_LS_CNN_trained = np.empty((0, H_data.shape[2], H_data.shape[3]))  # complex values
    X_LI_GAN_trained = np.empty((0, H_data.shape[2], H_data.shape[3]))
    X_LS_GAN_trained = np.empty((0, H_data.shape[2], H_data.shape[3]))
    
    X_LI_CNN_transfer = np.empty((0, H_data.shape[2], H_data.shape[3]))  # Shape (0, 612, 14)
    X_LS_CNN_transfer = np.empty((0, H_data.shape[2], H_data.shape[3]))  # complex values
    X_LI_GAN_transfer = np.empty((0, H_data.shape[2], H_data.shape[3]))
    X_LS_GAN_transfer = np.empty((0, H_data.shape[2], H_data.shape[3]))

    # Test the dataloader
    for batch_idx, batch in enumerate(dataloader):
        h, x, noise = batch
            # h == tensor [32,2,612,14] , x == [32,2,612,14], noise == [32,2,612,14]
        # creating complex values
        h_complex = torch.complex(h[:,0,:,:], h[:,1,:,:])
        x_complex = torch.complex(x[:,0,:,:], x[:,1,:,:])
        X_orig = np.concatenate((X_orig, x_complex.cpu().numpy()), axis=0)
        n_complex = torch.complex(noise[:,0,:,:], noise[:,1,:,:])
        Y_noise = h_complex * x_complex + n_complex # Element-wise multiplication # 32, 612, 14
        # print(f"Batch shapes: H: {h.shape}, X: {x.shape}, Noise: {noise.shape}")
        H_equalized, H_linear =  helperLinearInterp(Y_noise, pilot_Indices, pilot_Symbols)
        
        X_linear = Y_noise/H_linear
        X_LS_LI = np.concatenate((X_LS_LI, X_linear.cpu().numpy()), axis=0)
        
        H_LS  = torch.cat((torch.tensor(H_equalized.real).unsqueeze(1), torch.tensor(H_equalized.imag).unsqueeze(1)), dim=1)  # Shape: [32, 2, 612, 14]
        H_LI  = torch.cat((torch.tensor(H_linear.real).unsqueeze(1), torch.tensor(H_linear.imag).unsqueeze(1)), dim=1)  # Shape: [32, 2, 612, 14]

        # normalized data
        H_LS_normd, test_LS_min, test_LS_max = minmaxScaler(H_LS)
        H_LI_normd, test_LI_min, test_LI_max = minmaxScaler(H_LI)
            # testData_LS_normd = [32, 2, 612, 14] tensor, cpu 
            # test_LS_min = [32, 2] tensor, device  -- min of [32, real-imag]
        
        min_maxLS = torch.stack((test_LS_min, test_LS_max), dim=1)
        min_maxLI = torch.stack((test_LI_min, test_LI_max), dim=1)
            # [32, 2, 2] -- [:, 0, 0] = min, real       [:, 0, 1] = min, imag 
            #               [:, 1, 0] = max, real       [:, 1, 1] = max, imag
            
        LS_inputs_real = H_LS_normd[:,0,:,:].unsqueeze(1).to(device)
        LS_inputs_imag = H_LS_normd[:,1,:,:].unsqueeze(1).to(device)
        
        LI_inputs_real = H_LI_normd[:,0,:,:].unsqueeze(1).to(device)
        LI_inputs_imag = H_LI_normd[:,1,:,:].unsqueeze(1).to(device)

        ##### Trained-only models
        ##------------- LS+LI+CNN
        X_LI_CNN_trained_temp = helperEqualX(Y_noise, min_maxLI, LI_CNN_trained, LI_inputs_real, LI_inputs_imag, device)
        X_LI_CNN_trained_temp = X_LI_CNN_trained_temp.detach().cpu().numpy()
        X_LI_CNN_trained = np.concatenate((X_LI_CNN_trained, X_LI_CNN_trained_temp), axis=0)
        ## ------------ LS+CNN
        X_LS_CNN_trained_temp = helperEqualX(Y_noise, min_maxLS, LS_CNN_trained, LS_inputs_real, LS_inputs_imag, device)
        X_LS_CNN_trained_temp = X_LS_CNN_trained_temp.detach().cpu().numpy()
        X_LS_CNN_trained = np.concatenate((X_LS_CNN_trained, X_LS_CNN_trained_temp), axis=0)
        ## ------------ LS+LI+GAN
        X_LI_GAN_trained_temp = helperEqualX(Y_noise, min_maxLI, LI_GAN_trained, LI_inputs_real, LI_inputs_imag, device)
        X_LI_GAN_trained_temp = X_LI_GAN_trained_temp.detach().cpu().numpy()
        X_LI_GAN_trained = np.concatenate((X_LI_GAN_trained, X_LI_GAN_trained_temp), axis=0)
        ## ------------ LS+GAN
        X_LS_GAN_trained_temp = helperEqualX(Y_noise, min_maxLS, LS_GAN_trained, LS_inputs_real, LS_inputs_imag, device)
        X_LS_GAN_trained_temp = X_LS_GAN_trained_temp.detach().cpu().numpy()
        X_LS_GAN_trained = np.concatenate((X_LS_GAN_trained, X_LS_GAN_trained_temp), axis=0)
        
        ##### Transfer models
        ##------------- LS+LI+CNN
        X_LI_CNN_transfer_temp = helperEqualX(Y_noise, min_maxLI, LI_CNN_transfer, LI_inputs_real, LI_inputs_imag, device)
        X_LI_CNN_transfer_temp = X_LI_CNN_transfer_temp.detach().cpu().numpy()
        X_LI_CNN_transfer = np.concatenate((X_LI_CNN_transfer, X_LI_CNN_transfer_temp), axis=0)
        ## ------------ LS+CNN
        X_LS_CNN_transfer_temp = helperEqualX(Y_noise, min_maxLS, LS_CNN_transfer, LS_inputs_real, LS_inputs_imag, device)
        X_LS_CNN_transfer_temp = X_LS_CNN_transfer_temp.detach().cpu().numpy()
        X_LS_CNN_transfer = np.concatenate((X_LS_CNN_transfer, X_LS_CNN_transfer_temp), axis=0)
        ## ------------ LS+LI+GAN
        X_LI_GAN_transfer_temp = helperEqualX(Y_noise, min_maxLI, LI_GAN_transfer, LI_inputs_real, LI_inputs_imag, device)
        X_LI_GAN_transfer_temp = X_LI_GAN_transfer_temp.detach().cpu().numpy()
        X_LI_GAN_transfer = np.concatenate((X_LI_GAN_transfer, X_LI_GAN_transfer_temp), axis=0)
        ## ------------ LS+GAN
        X_LS_GAN_transfer_temp = helperEqualX(Y_noise, min_maxLS, LS_GAN_transfer, LS_inputs_real, LS_inputs_imag, device)
        X_LS_GAN_transfer_temp = X_LS_GAN_transfer_temp.detach().cpu().numpy()
        X_LS_GAN_transfer = np.concatenate((X_LS_GAN_transfer, X_LS_GAN_transfer_temp), axis=0)   
        
        # end of batch loop dataloader
        

    # save (X_est, X_orig) to .mat file of each SNR
    data_dict = {
        "X_orig": X_orig,
        "H_LS_LI": X_LS_LI,
        "X_LI_CNN_trained": X_LI_CNN_trained,
        "X_LS_CNN_trained": X_LS_CNN_trained,
        "X_LI_GAN_trained": X_LI_GAN_trained,
        "X_LS_GAN_trained": X_LS_GAN_trained,
        "X_LI_CNN_transfer": X_LI_CNN_transfer,
        "X_LS_CNN_transfer": X_LS_CNN_transfer,
        "X_LI_GAN_transfer": X_LI_GAN_transfer,
        "X_LS_GAN_transfer": X_LS_GAN_transfer
    }
    sio.savemat('data/dataSNR' +str(snr) +'dB.mat', data_dict)
    
    # end of SNR loop


snr -25/30


/home/thien/Hprediction/one_shot_Hest_cleanver/Torch_code/BER_calulation/helper_BER.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = (torch.load(model_path 

snr -20/30
snr -15/30
snr -10/30
snr -5/30
snr 0/30


/home/thien/Hprediction/one_shot_Hest_cleanver/Torch_code/BER_calulation/helper_BER.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = (torch.load(model_path

snr 5/30
snr 10/30
snr 15/30
snr 20/30
snr 25/30
snr 30/30
